In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, execute
from qiskit.aqua.components.optimizers import GSLS, L_BFGS_B, SLSQP, TNC, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES
import math
import random
import numpy as np
from numpy import pi
import time
import itertools
import pickle

## Auxiliary functions

In [2]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def prepare_initial_state(theta, phase):
    """ Prepare initial state """
    return (math.cos(theta),
        (math.sin(theta) * math.e**(1j * phase)))

In [4]:
def convert_counts_to_final_result(counts) -> int:
    """ Convert the execution result to the final measured value: 0 or 1 """
    if "0" in counts:
        return 0
    return 1

In [5]:
def set_random_lambda(lambda0, lambda1):
    """ return a random choice from lambda0 and lambda1 with the correspondent key name """
    lambda_value = random.choice([lambda0, lambda1])
    if lambda_value == lambda0:
        return lambda_value, 0
    return lambda_value, 1

In [6]:
def compute_damping_channel(theta, phase, one_lambda, phi_rx, phi_ry, backend) -> int:
    """ one-time execution of the amplitude damping circuit using the passed parameters 
        Returns: the execution measured result: either 0 or 1
    """
    qreg_q = QuantumRegister(2, 'q')
    creg_c = ClassicalRegister(1, 'c')

    initial_state = prepare_initial_state(theta, phase)

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize([initial_state[0],
                        initial_state[1]], qreg_q[0])
    circuit.reset(qreg_q[1])
    circuit.cry(2 * one_lambda, qreg_q[0], qreg_q[1])
    circuit.cx(qreg_q[1], qreg_q[0])
    circuit.rx(phi_rx, qreg_q[0])
    circuit.ry(phi_ry, qreg_q[0])
    circuit.measure(qreg_q[0], creg_c[0])

    counts = execute(circuit, backend, shots=1).result().get_counts(circuit)
    return convert_counts_to_final_result(counts)

In [7]:
def compute_random_damping_channel(configuration, backend):
    """ run a one-time execution on the given backend device randomly selectiong the lambda to be used and return the measured result """
    lambda_value, lambda_used = set_random_lambda(configuration['lambda0'], configuration['lambda1'])
    result = compute_damping_channel(configuration['theta'], configuration['phase'], lambda_value, configuration['phi_rx'], configuration['phi_ry'], backend)

    return {
        "result": result,
        "lambda_used": lambda_used
    }

In [8]:
def guess_lambda_used(real_measured_result):
    """ Decides which lambda was used on the real execution.
        It is a silly guess.
        It returns the same lambda used as the measured result
    """
    return real_measured_result

In [9]:
def check_value(real_lambda, guess_lambda): 
    if real_lambda == guess_lambda: return 1
    return 0

In [10]:
def play_and_guess_one_case(configuration, backend):
    """ Execute a real execution with a random lambda from the two passed, 
        guess which one was used on the exection and
        check the result.
        Returns 1 on success (it was a correct guess) or 0 on fail (it was an incorrect guess) 
    """
    real_result = compute_random_damping_channel(configuration, backend)
    guess_lambda = guess_lambda_used(real_result['result'])
    return check_value(real_result['lambda_used'], guess_lambda)

In [11]:
def get_combinations_two_etas_without_repeats(input_list_eta):
    """ from a given list of attenuation angles (etas) create a 
        list of all combinatorial pairs of possible etas
        without repeats (order does not matter).
        For us it is the same testing first eta 0.1 and second eta 0.2 
        than first eta 0.2 and second eta 0.1
    """
    list_eta = input_list_eta
     # when there is only one element, we add the same element
    if len(list_eta) == 1:
        list_eta.append(list_eta[0])
    # get combinations of two etas without repeats
    return list(itertools.combinations(list_eta, 2))

## Creating our cost function to be minimized (minimizing the error)

In [12]:
def play_one_configuration(configuration, backend, plays=1000):
    """ Play the play_and_guess game for the number of plays for the configuration of input parameters.
        Returns the average success probability for all the plays with the given configuration. 
    """
    success_counts = 0
    for play in range(plays):
        success_counts += play_and_guess_one_case(configuration, backend)
    return success_counts/plays

In [13]:
def cost_function(params):
    """ Computes the cost of running a specific configuration.
        Cost is computed as 1 (perfect probability) - given probability [0,1]
    """
    backend = backend=Aer.get_backend('qasm_simulator')

    configuration = {
        "theta": params[0],
        "phase": params[1],
        "phi_rx": params[2],
        "phi_ry": params[3],
        "lambda0": GLOBAL_LAMBDA0,
        "lambda1": GLOBAL_LAMBDA1
    }
    return 1 - play_one_configuration(configuration, backend, plays=100)

## Parameter Optimization for the best configuration

In [14]:
def compute_best_configuration( optimizer_algorithms=['CRS'], 
                                list_optimizer_iterations=[1000], 
                                list_step_size=[pi/10], 
                                list_iterations_per_algorithm=[1],
                                initial_params = [0, 0, 0, 0]):
    """ Find out the best configuration with a global pair of lambdas (channels) trying out a list of specified optimizat algorithm """
    # Create the initial parameters and set the bounds
    #initial_params = [0, 0, 0, 0]
    variable_bounds = [
                        (0, pi/2), # theta
                        (0, 2*pi), # phase
                        (0, 2*pi), # rx
                        (0, 2*pi)  # ry
                        ]
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""

    for optimizer_algorithm, optimizer_iterations, step_size, iterations in zip(optimizer_algorithms, 
                                                                    list_optimizer_iterations, 
                                                                    list_step_size, 
                                                                    list_iterations_per_algorithm):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'GSLS':
            # optimizer = GSLS()
            optimizer = GSLS(maxiter=optimizer_iterations, max_eval=optimizer_iterations, initial_step_size=step_size)
        if optimizer_algorithm == 'L_BFGS_B':
            # optimizer = L_BFGS_B()
            optimizer = L_BFGS_B(maxfun=optimizer_iterations, maxiter=optimizer_iterations, epsilon=step_size)
        if optimizer_algorithm == 'SLSQP':
            # optimizer = SLSQP()
            optimizer = SLSQP(maxiter=optimizer_iterations,  eps=step_size)
        if optimizer_algorithm == 'TNC':
            # optimizer = TNC()
            optimizer = TNC(maxiter=optimizer_iterations, eps=step_size)
        if optimizer_algorithm == 'BOBYQA':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'IMFIL':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'SNOBFIT':
            optimizer = BOBYQA(maxiter=optimizer_iterations)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=optimizer_iterations)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=optimizer_iterations)

        
        for i in range(iterations):
            ret = optimizer.optimize(num_vars=len(initial_params), 
                                    objective_function=cost_function, 
                                    variable_bounds=variable_bounds,
                                    initial_point=initial_params)
            print("Best Average Probability:", 1 - ret[1])
            if (1-ret[1]) > best_probability:
                best_probability = 1-ret[1]
                best_configuration = ret[0]
                best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Parameters Found: " + u"\u03B8" + " = " + str(int(math.degrees(best_configuration[0]))) + u"\u00B0" +
                            ", Phase = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "rx = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" + 
                            ", " + u"\u03D5" + "ry = " + str(int(math.degrees(best_configuration[3]))) + u"\u00B0" +
                            ", " + u"\u03BB" + u"\u2080" + " = " + str(GLOBAL_LAMBDA0) + 
                            ", " + u"\u03BB" + u"\u2081" + " = " + str(GLOBAL_LAMBDA1))
    return {
        "best_optimizer_algorithm": best_optimizer_algorithm,
        "best_probability": best_probability,
        "best_configuration": best_configuration
    }


## Find out the best configuration ($\theta, phase, rx, ry$) and success probability for each pair of $\lambda$ attenuation levels from 0 to 1

In [16]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

#optimizer_algorithms = ['TNC']
#list_optimizer_iterations=[10]
#list_step_size=[pi/10]
#list_iterations_per_algorithm=[1]
optimizer_algorithms = ['CRS', 'DIRECT_L', 'DIRECT_L_RAND', 'ESCH', 'ISRES']
list_optimizer_iterations=[1000, 1000, 1000, 1000, 1000]
list_step_size=[pi/10, pi/10, pi/10, pi/10, pi/10]
list_iterations_per_algorithm=[1, 1, 1, 1, 1]
# optimizer_algorithms = ['CRS', 'DIRECT_L', 'DIRECT_L_RAND', 'ESCH', 'ISRES']
# list_optimizer_iterations=[1000, 1000, 1000, 1000, 1000]
# list_step_size=[pi/10, pi/10, pi/10, pi/10, pi/10]
# list_iterations_per_algorithm=[1, 1, 1, 1, 1]
# optimizer_algorithms = ['GSLS', 'L_BFGS_B', 'SLSQP', 'TNC']
# list_optimizer_iterations=[10000, 1000, 100, 100]
# list_step_size=[pi/10, pi/10, pi/10, pi/10]
# list_iterations_per_algorithm=[1, 1, 1, 1]
two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []
initial_params = [0, 0, 0, 0]

total_eta_pairs = len(two_pair_etas)
clones = 12 # Number of parallel calculators (VMs)
clon = 0    # VM Calculator Number

if clones > 1:
    eta_pair_ini = np.ceil(clon*total_eta_pairs/clones)
    eta_pair_fin = min(np.ceil((clon+1)*total_eta_pairs/clones)-1, total_eta_pairs-1)
else:
    eta_pair_ini = 0
    eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")

for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    if eta_pair_ini > two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    elif eta_pair_fin < two_pair_etas.index(eta_pair):
        probabilities.append(prob)
        configurations.append(config)
        best_algorithm.append('NA')
    else:
        GLOBAL_LAMBDA0=eta_pair[0]
        GLOBAL_LAMBDA1=eta_pair[1]
        result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm,
                                        initial_params=initial_params)
        probabilities.append(result['best_probability'])
        configurations.append(result['best_configuration'])
        best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))

end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.64
Parameters Found: θ = 20°, Phase = 306°, ϕrx = 231°, ϕry = 60°, λ₀ = 0.0, λ₁ = 0.7853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.76
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.76
Parameters Found: θ = 29°, Phase = 336°, ϕrx = 190°, ϕry = 286°, λ₀ = 0.0, λ₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.75
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.75
Parameters Found: θ = 50°, Phase = 116°, ϕrx = 304°, ϕry = 169°, λ₀ = 0.7853981633974483, λ₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  0
total minutes of executi

In [82]:
""" Save results to disk """
# save_results_to_disk(result, name='20210331_C1_Clon00')

In [24]:
""" Load results from a file """
loaded_results = load_results_from_file(name='20210331_results_CRS_clons')


In [26]:
# create the channels to be used
# from 0 to 1 in steps of 0.1, and adding the 1 as the last value
etas = np.append(np.arange(0, pi/2, pi/2/20), pi/2)

optimizer_algorithms = ['CRS', 'DIRECT_L', 'DIRECT_L_RAND', 'ESCH', 'ISRES']
list_optimizer_iterations=[1, 1, 1, 1, 1]
list_step_size=[pi/10, pi/10, pi/10, pi/10, pi/10]
list_iterations_per_algorithm=[1, 1, 1, 1, 1]

two_pair_etas = get_combinations_two_etas_without_repeats(etas)
probabilities = []
configurations = []
best_algorithm = []

total_eta_pairs = len(two_pair_etas)
eta_pair_ini = 0
eta_pair_fin = total_eta_pairs-1

program_start_time = time.time()
print("Starting the execution")
i=0
for eta_pair in two_pair_etas:
    start_time = time.time()
    prob = 0
    config = np.array([0.0, 0.0, 0.0])
    GLOBAL_LAMBDA0=eta_pair[0]
    GLOBAL_LAMBDA1=eta_pair[1]
    initial_params=[loaded_results['configurations'][i][0], loaded_results['configurations'][i][1], loaded_results['configurations'][i][2], loaded_results['configurations'][i][3]]
    result = compute_best_configuration(optimizer_algorithms=optimizer_algorithms,
                                        list_optimizer_iterations=list_optimizer_iterations,
                                        list_step_size=list_step_size, 
                                        list_iterations_per_algorithm=list_iterations_per_algorithm,
                                        initial_params=initial_params)
    probabilities.append(result['best_probability'])
    configurations.append(result['best_configuration'])
    best_algorithm.append(result['best_optimizer_algorithm'])
    end_time = time.time()
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time)/60)))
    print("total minutes taken so far: ", int(np.round((end_time - program_start_time)/60)))
    i=i+1
end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time)/60)))
print("All guesses have been calculated")
print(f'Total pair of etas tested: {len(two_pair_etas)}')

result = {
    "lambda_pairs": two_pair_etas,
    "best_algorithm": best_algorithm,
    "probabilities": probabilities,
    "configurations": configurations
}

Starting the execution
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.59
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.52
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.63
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.48
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.54
Final Best Optimizer Algorithm:  DIRECT_L_RAND
Final Best Average Probability: 0.63
Parameters Found: θ = 45°, Phase = 180°, ϕrx = 180°, ϕry = 180°, λ₀ = 0.0, λ₁ = 0.07853981633974483
total minutes taken this pair of etas:  0
total minutes taken so far:  0
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.47
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.56
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.52
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.49
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.48
Final Best Opt

Best Average Probability: 0.78
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.92
Parameters Found: θ = 81°, Phase = 87°, ϕrx = 173°, ϕry = 0°, λ₀ = 0.0, λ₁ = 1.0995574287564276
total minutes taken this pair of etas:  0
total minutes taken so far:  2
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.94
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.44999999999999996
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.3400000000000001
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.86
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.88
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.94
Parameters Found: θ = 90°, Phase = 0°, ϕrx = 185°, ϕry = 360°, λ₀ = 0.0, λ₁ = 1.1780972450961724
total minutes taken this pair of etas:  0
total minutes taken so far:  2
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.95
Analyzing Optimizer Algorithm:  D

Best Average Probability: 0.64
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.48
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.44999999999999996
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.65
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.65
Parameters Found: θ = 89°, Phase = 355°, ϕrx = 360°, ϕry = 192°, λ₀ = 0.07853981633974483, λ₁ = 0.7068583470577035
total minutes taken this pair of etas:  0
total minutes taken so far:  4
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.74
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.54
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.41999999999999993
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.78
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.73
Final Best Optimizer Algorithm:  ESCH

Best Average Probability: 0.59
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.61
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.52
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.61
Parameters Found: θ = 51°, Phase = 122°, ϕrx = 135°, ϕry = 23°, λ₀ = 0.15707963267948966, λ₁ = 0.3141592653589793
total minutes taken this pair of etas:  0
total minutes taken so far:  6
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.55
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.55
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.56
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.59
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.49
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.59
Parameters Found: θ = 61°, Phase = 130°, ϕrx = 360°, ϕry = 160°, λ₀ = 0.15707963267948966, λ₁ = 0.39269908169872414
total minutes taken this p

Best Average Probability: 0.93
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.95
Parameters Found: θ = 90°, Phase = 360°, ϕrx = 360°, ϕry = 185°, λ₀ = 0.15707963267948966, λ₁ = 1.335176877775662
total minutes taken this pair of etas:  0
total minutes taken so far:  7
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.98
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.48
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.66
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.99
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.92
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.99
Parameters Found: θ = 90°, Phase = 37°, ϕrx = 165°, ϕry = 0°, λ₀ = 0.15707963267948966, λ₁ = 1.413716694115407
total minutes taken this pair of etas:  0
total minutes taken so far:  7
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.95
Analyzing Optimizer Algorith

...

Best Average Probability: 0.57
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.55
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.59
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.66
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.66
Parameters Found: θ = 67°, Phase = 54°, ϕrx = 179°, ϕry = 351°, λ₀ = 0.7068583470577035, λ₁ = 1.0210176124166828
total minutes taken this pair of etas:  0
total minutes taken so far:  19
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.64
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.49
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.58
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.62
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.62
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability:

Best Average Probability: 0.47
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.65
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.69
Parameters Found: θ = 90°, Phase = 360°, ϕrx = 152°, ϕry = 0°, λ₀ = 0.7853981633974483, λ₁ = 1.2566370614359172
total minutes taken this pair of etas:  0
total minutes taken so far:  21
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.69
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.38
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.66
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.72
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.64
Final Best Optimizer Algorithm:  ESCH
Final Best Average Probability: 0.72
Parameters Found: θ = 90°, Phase = 360°, ϕrx = 0°, ϕry = 170°, λ₀ = 0.7853981633974483, λ₁ = 1.335176877775662
total minutes taken this pair of 

Best Average Probability: 0.56
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.62
Parameters Found: θ = 84°, Phase = 297°, ϕrx = 207°, ϕry = 0°, λ₀ = 0.8639379797371931, λ₁ = 1.5707963267948966
total minutes taken this pair of etas:  0
total minutes taken so far:  23
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.42999999999999994
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.57
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.52
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.51
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.48
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.57
Parameters Found: θ = 15°, Phase = 180°, ϕrx = 180°, ϕry = 180°, λ₀ = 0.9424777960769379, λ₁ = 1.0210176124166828
total minutes taken this pair of etas:  0
total minutes taken so far:  23
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.53
Analyzi

Best Average Probability: 0.68
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.38
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.56
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.62
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.71
Final Best Optimizer Algorithm:  ISRES
Final Best Average Probability: 0.71
Parameters Found: θ = 90°, Phase = 270°, ϕrx = 0°, ϕry = 163°, λ₀ = 1.0210176124166828, λ₁ = 1.4922565104551517
total minutes taken this pair of etas:  0
total minutes taken so far:  25
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.54
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.51
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.53
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.52
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.66
Final Best Optimizer Algorithm:  ISRES
Final Best Average Probabilit

Best Average Probability: 0.58
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.45999999999999996
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.52
Final Best Optimizer Algorithm:  DIRECT_L_RAND
Final Best Average Probability: 0.58
Parameters Found: θ = 45°, Phase = 180°, ϕrx = 180°, ϕry = 60°, λ₀ = 1.2566370614359172, λ₁ = 1.335176877775662
total minutes taken this pair of etas:  0
total minutes taken so far:  27
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.57
Analyzing Optimizer Algorithm:  DIRECT_L
Best Average Probability: 0.65
Analyzing Optimizer Algorithm:  DIRECT_L_RAND
Best Average Probability: 0.5
Analyzing Optimizer Algorithm:  ESCH
Best Average Probability: 0.57
Analyzing Optimizer Algorithm:  ISRES
Best Average Probability: 0.57
Final Best Optimizer Algorithm:  DIRECT_L
Final Best Average Probability: 0.65
Parameters Found: θ = 15°, Phase = 180°, ϕrx = 180°, ϕry = 180°, λ₀ = 1.2566370614359172, λ₁ = 1.413716694115407
tot

In [27]:
""" Save results to disk """
save_results_to_disk(result, name='20210401_OneShot_ConfigImproved')